In [ ]:
!tar -xvf ./data/hebtb.tar.gz

In [1]:
import pandas as pd
import numpy as np

## Files

In [2]:
filepath_spmrl_dev = './data/spmrl-treebank/dev_hebtb-gold.conll'
filepath_spmrl_train = './data/spmrl-treebank/train_hebtb-gold.conll'
filepath_spmrl_test = './data/spmrl-treebank/test_hebtb-gold.conll'

filepath_ud_dev = './data/ud-treebank/he_htb-ud-dev.conllu'
filepath_ud_train = './data/ud-treebank/he_htb-ud-train.conllu'
filepath_ud_test = './data/ud-treebank/he_htb-ud-test.conllu'


In [3]:
def suit_for_pandas(filepath):
    treebank = []
    source = open(filepath, 'r')
    for line in source.readlines():
        if len(line.split('\t')) == 10:
            treebank.append(tuple(line.strip().split('\t')))
        elif len(line.split('\t')) == 1:
            treebank.append((line.strip(), '', '', '', '', '', '', '', '', ''))
    if 'spmrl' in filepath:
        return pd.DataFrame(data = treebank, columns=['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC'])
    else:
        return pd.DataFrame(data = treebank, columns=['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC'])



In [4]:
ud_dev = suit_for_pandas(filepath_ud_dev)
spmrl_dev = suit_for_pandas(filepath_spmrl_dev)

In [ ]:
ud_train = suit_for_pandas(filepath_ud_train)
spmrl_train = suit_for_pandas(filepath_spmrl_train)

In [5]:
ud_dev['sent_id'] = ''
spmrl_dev['sent_id'] = ''
spmrl_dev['ID'] = spmrl_dev['ID'].apply(lambda x: int(x) if x.isdigit() else 0)

In [9]:
spmrl_dev['sent_id'] = ''
spmrl_dev['ID'] = spmrl_dev['ID'].apply(lambda x: int(x) if x.isdigit() else 0)
sentence_id(spmrl_dev, 'spmrl')

In [ ]:
ud_train['sent_id'] = ''
sentence_id(ud_train, 'ud')

In [6]:
def sentence_id(df, tb):
    if tb=='ud':
        sent_id = 0
        for i, row in df.iterrows():
            if '# sent_id' in row['ID']:
                sent_id += 1
            else:
                df.at[i, 'sent_id'] = sent_id
    elif tb=='spmrl':
        sent_id = 1
        for i, row in df.iterrows():
            if row['ID'] == 0:
                df.at[i, 'sent_id'] = 0 
            try:
                if df.loc[i]['ID'] > df.loc[i-1]['ID']:
                    df.at[i, 'sent_id'] = sent_id
                elif type(df.loc[i-1]['ID']) == str:
                    continue
                else:
                    sent_id += 1
                    df.at[i, 'sent_id'] = sent_id
            except KeyError as e:
                df.at[i, 'sent_id'] = 1
            except TypeError as e:
                print(df.loc[i]['ID'],df.loc[i-1]['ID'], e)
                
sentence_id(spmrl_dev, 'spmrl')
sentence_id(ud_dev, 'ud')

## Inspection

In [34]:
ud_dev[ud_dev['FORM'].str.contains('כל')]

,ID,FORM,LEMMA,UPOS,XPOS,FEATS,HEAD,DEPREL,DEPS,MISC,sent_id
217,16-17,שתכלול,_,_,_,_,_,_,_,_,6
219,17,תכלול,כלל,VERB,VERB,Gender=Fem|HebBinyan=PAAL|Number=Sing|Person=3...,7,acl:relcl,_,_,6
505,35-36,בכל,_,_,_,_,_,_,_,_,13
507,36,כל,כול,DET,DET,Definite=Cons,37,det,_,_,13
837,25-26,שכל,_,_,_,_,_,_,_,_,23
839,26,כל,כול,DET,DET,Definite=Cons,27,det,_,_,23
927,17,כל,כול,DET,DET,Definite=Cons,18,det,_,_,25
1066,32,כלל,כלל,ADV,ADV,_,33,advmod,_,_,27
1074,1-2,לכל,_,_,_,_,_,_,_,_,28
1076,2,כל,כול,DET,DET,Definite=Cons,3,det,_,_,28


In [35]:
ud_dev[ud_dev['sent_id'] == 13].head(60)#['FEATS'].unique()

,ID,FORM,LEMMA,UPOS,XPOS,FEATS,HEAD,DEPREL,DEPS,MISC,sent_id
463,"# text = ח""כ רן כהן (רץ) אמר כי על הוועדה לפנו...",,,,,,,,,,13
464,1,"ח""כ","ח""ך",NOUN,NOUN,Abbr=Yes|Gender=Masc|Number=Sing,7,dep,_,_,13
465,2,רן,רן,PROPN,PROPN,_,1,flat:name,_,_,13
466,3,כהן,כהן,PROPN,PROPN,_,1,flat:name,_,_,13
467,4,(,(,PUNCT,PUNCT,_,5,punct,_,SpaceAfter=No,13
468,5,רץ,רצ,PROPN,PROPN,_,1,appos,_,SpaceAfter=No,13
469,6,),),PUNCT,PUNCT,_,5,punct,_,_,13
470,7,אמר,אמר,VERB,VERB,Gender=Masc|HebBinyan=PAAL|Number=Sing|Person=...,0,root,_,_,13
471,8,כי,כי,SCONJ,SCONJ,_,11,mark,_,_,13
472,9,על,על,ADP,ADP,_,11,case,_,_,13


In [ ]:
spmrl_dev[spmrl_dev['sent_id'] == 4539].head(60)#['FEATS'].unique()

In [32]:
spmrl_dev[spmrl_dev['XPOS']== "DTT"]#['FEATS'].unique()

,ID,FORM,LEMMA,UPOS,XPOS,FEATS,HEAD,DEPREL,DEPS,MISC,sent_id
395,36,כל,כול,DTT,DTT,_,37,det,_,_,13
561,12,מספר,מספר,DTT,DTT,_,14,det,_,_,20
584,10,אף,אף,DTT,DTT,_,11,det,_,_,21
637,26,כל,כול,DTT,DTT,_,27,det,_,_,23
705,17,כל,כול,DTT,DTT,_,18,det,_,_,25
820,2,כל,כול,DTT,DTT,_,3,det,_,_,28
851,33,כל,כול,DTT,DTT,_,34,det,_,_,28
1132,1,כל,כול,DTT,DTT,_,2,det,_,_,40
1158,27,כל,כול,DTT,DTT,_,26,mwe,_,_,40
1161,30,כל,כול,DTT,DTT,_,31,det,_,_,40


In [ ]:
spmrl_train[(spmrl_train['XPOS'] == 'DTT') & ~(spmrl_train['FEATS'].str.contains('suf_gen', na=False) )]#['FEATS'].unique()

In [ ]:
spmrl_train[spmrl_train['sent_id'] == 106]

In [ ]:
ud_train[(ud_train['LEMMA'] == 'או') ]#['XPOS'].unique()

In [ ]:
ud_dev = suit_for_pandas('./data/he_htb-ud-dev.conllu')
ud_dev.columns = ['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC']

In [ ]:
spmrl_dev[spmrl_dev['XPOS'].str.contains('NN_S_PP', na=False)]

In [ ]:
ud_dev[(ud_dev['XPOS'].str.contains('BN', na=False)) & (ud_dev['FEATS'].str.contains('HIFIL', na=False))]

In [ ]:
spmrl_dev.to_csv('./data/spmrl-treebank/numbered_dev.csv')

## Segmentation

In [49]:
segmented_spmrl_df = pd.DataFrame(columns=['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC', 'sent_id'])

In [26]:
pronouns = {
     'suf_gen=F|suf_gen=M|suf_num=P|suf_per=1': '_אנחנו',
     'suf_gen=F|suf_gen=M|suf_num=S|suf_per=1': '_אני',
     'suf_gen=M|suf_num=S|suf_per=2': '_אתה',
     'suf_gen=F|suf_num=S|suf_per=2': '_את',
     'suf_gen=M|suf_num=P|suf_per=2': '_אתם',
     'suf_gen=F|suf_num=P|suf_per=2': '_אתן',
     'suf_gen=F|suf_num=P|suf_per=3': '_הן',
     'suf_gen=F|suf_num=S|suf_per=3': '_היא',
     'suf_gen=M|suf_num=P|suf_per=3': '_הם',
     'suf_gen=M|suf_num=S|suf_per=3': '_הוא'
}

pronouns

{'suf_gen=F|suf_gen=M|suf_num=P|suf_per=1': '_אנחנו',
 'suf_gen=F|suf_gen=M|suf_num=S|suf_per=1': '_אני',
 'suf_gen=M|suf_num=S|suf_per=2': '_אתה',
 'suf_gen=F|suf_num=S|suf_per=2': '_את',
 'suf_gen=M|suf_num=P|suf_per=2': '_אתם',
 'suf_gen=F|suf_num=P|suf_per=2': '_אתן',
 'suf_gen=F|suf_num=P|suf_per=3': '_הן',
 'suf_gen=F|suf_num=S|suf_per=3': '_היא',
 'suf_gen=M|suf_num=P|suf_per=3': '_הם',
 'suf_gen=M|suf_num=S|suf_per=3': '_הוא'}

In [50]:
for i, row in spmrl_dev.iterrows():
    if row['XPOS'] == 'NN_S_PP' or row['XPOS'] == 'S_PP':
        noun_feats = "|".join([x for x in row['FEATS'].split("|") if 'suf' not in x])
        suffix_feats = "|".join([x for x in row['FEATS'].split("|") if 'suf' in x])
        clean_suffix_feats = "|".join([x.replace("suf_", "") for x in row['FEATS'].split("|") if 'suf' in x])

        segmented_spmrl_df = segmented_spmrl_df.append({'ID': row['ID'], 'FORM': row['LEMMA'] + '_', 'LEMMA': row['LEMMA'],  'UPOS': 'NOUN', 
                                                        'XPOS': 'NOUN','FEATS': 'Definite=Def|' + noun_feats, 'HEAD': row['HEAD'], 
                                                        'DEPREL': row['DEPREL'], 'DEPS': row['DEPS'], 'MISC': row['MISC'],'sent_id': row['sent_id']}, ignore_index=True)
        
        segmented_spmrl_df = segmented_spmrl_df.append({'ID': 0, 'FORM': '_של_', 'LEMMA': 'של',  'UPOS': 'ADP', 
                                                        'XPOS': 'ADP','FEATS': '_', 'HEAD': int(row['ID']) + 2, 
                                                        'DEPREL': 'case:gen', 'DEPS': row['DEPS'], 'MISC': row['MISC'],'sent_id': row['sent_id']}, ignore_index=True)

        segmented_spmrl_df = segmented_spmrl_df.append({'ID': 0, 'FORM': pronouns[suffix_feats], 'LEMMA': 'הוא',  'UPOS': 'PRON', 
                                                        'XPOS': 'PRON','FEATS': "Case=Gen|" + clean_suffix_feats + "|PronType=Prs", 'HEAD': int(row['ID']) + 2, 
                                                        'DEPREL': 'nmod:poss', 'DEPS': row['DEPS'], 'MISC': row['MISC'],'sent_id': row['sent_id']}, ignore_index=True)
        
    elif row['XPOS'] == 'S_PRN':
        segmented_spmrl_df.at[i-1, 'XPOS'] = 'ADP'
        segmented_spmrl_df.at[i-1, 'FEATS'] = 'Case=Gen'
        segmented_spmrl_df = segmented_spmrl_df.append({'ID': row['ID'], 'LEMMA': row['LEMMA'],  'UPOS': 'PRON', 
                            'XPOS': 'PRON', 'FEATS': segmented_spmrl_df.loc[i-1]['FEATS'] + '|PronType=Prs', 'HEAD': row['HEAD'], 
                            'DEPREL': row['DEPREL'], 'DEPS': row['DEPS'], 'MISC': row['MISC'],'sent_id': row['sent_id']}, ignore_index=True)

    elif row['XPOS'] == 'DTT' or row['XPOS'] == 'DT':
        if 'suf_' in row['FEATS']:
            segmented_spmrl_df = segmented_spmrl_df.append({'ID': row['ID'], 'LEMMA': row['LEMMA'],  'UPOS': 'NOUN', 
                                                        'XPOS': 'NOUN','FEATS': row['FEATS'], 'HEAD': row['HEAD'], 
                                                        'DEPREL': row['DEPREL'], 'DEPS': row['DEPS'], 'MISC': row['MISC'],'sent_id': row['sent_id']}, ignore_index=True)
        
            segmented_spmrl_df = segmented_spmrl_df.append({'ID': 0, 'FORM': "_" + pronouns[suffix_feats], 'LEMMA': 'הוא',  'UPOS': 'PRON', 
                                                        'XPOS': 'PRON','FEATS': "Case=Gen|"+clean_suffix_feats + "|PronType=Prs", 'HEAD': int(row['ID']) + 1, 
                                                        'DEPREL': 'nmod:poss', 'DEPS': row['DEPS'], 'MISC': row['MISC'],'sent_id': row['sent_id']}, ignore_index=True)
    
    elif row['XPOS'] == 'S_PRP':
        segmented_spmrl_df = segmented_spmrl_df.append({'ID': row['ID'], 'LEMMA': row['LEMMA'],  'UPOS': row['UPOS'], 
                                                        'XPOS': row['XPOS'],'FEATS': row['FEATS'] + "|PronType=Prs|Reflex=Yes", 'HEAD': row['HEAD'], 
                                                        'DEPREL': row['DEPREL'], 'DEPS': row['DEPS'], 'MISC': row['MISC'],'sent_id': row['sent_id']}, ignore_index=True)
    else:
        segmented_spmrl_df = segmented_spmrl_df.append(row, ignore_index=True)

In [52]:
segmented_spmrl_df[segmented_spmrl_df['XPOS'] == 'NOUN']#['FEATS'].unique()

,ID,FORM,LEMMA,UPOS,XPOS,FEATS,HEAD,DEPREL,DEPS,MISC,sent_id
140,26,הזמנה_,הזמנה,NOUN,NOUN,Definite=Def|gen=F|num=S,25,obj,_,_,5
338,22,העדפה_,העדפה,NOUN,NOUN,Definite=Def|gen=F|num=S,21,obj,_,_,11
407,2,דבר_,דבר,NOUN,NOUN,Definite=Def|gen=M|num=P,1,pobj,_,_,14
460,19,פרצוף_,פרצוף,NOUN,NOUN,Definite=Def|gen=M|num=S,18,pobj,_,_,15
542,16,מספר_,מספר,NOUN,NOUN,Definite=Def|gen=M|num=S,15,gobj,_,_,19
556,28,מקום_,מקום,NOUN,NOUN,Definite=Def|gen=M|num=S,27,pobj,_,_,19
664,1,מאמר_,מאמר,NOUN,NOUN,Definite=Def|gen=M|num=S,25,subj,_,_,24
692,27,יד_,יד,NOUN,NOUN,Definite=Def|gen=F|num=P,26,pobj,_,_,24
724,25,שם_,שם,NOUN,NOUN,Definite=Def|gen=M|num=S,24,pobj,_,_,25
888,22,משימה_,משימה,NOUN,NOUN,Definite=Def|gen=F|num=S,21,obj,_,_,29
